In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0,'../')
from pose_graph_partitioning.pose_graph import *
from pose_graph_partitioning.pose_graph_partitioning import *
from simulate_utils import ATE, align_posegraphs
from subprocess import PIPE, Popen
import copy

In [2]:
def call_d2pgo(input_data, output_data, agent_num, params, param_fixed={}, cmd="roslaunch d2pgo d2pgo_test_multi.launch "):
    command = f"{cmd} g2o_path:={input_data} output_path:={output_data} agent_num:={agent_num} "
    for k, v in params:
        command += f"{k}:={v[0]} "
    for k in param_fixed:
        v = param_fixed[k]
        command += f"{k}:={v} "
    p = Popen(command, shell=True, stdout=PIPE, stderr=PIPE)
    stdout, stderr = p.communicate()
    # return stdout, stderr
    # print(stdout.decode("utf-8"))
    return

def check_param_unique(params):
    cnt = 0
    for k, v in params:
        if len(v) > 1:
            return False, cnt
        cnt += 1
    return True, -1 

def print_res_to_table(res, sort_index="ATE_T"):
    import tabulate
    output_table = []
    #Add title
    if len(res) == 0:
        return []
    sort_values = []
    for row in res:
        output_table.append([])
        for k in row:
            v = row[k]
            if isinstance(v, np.ndarray) or isinstance(v, list):
                output_table[-1].append(v[0])
            else:
                if k == "ATE_T":
                    output_table[-1].append(f"{v:.3f}m")
                elif k == "ATE_rot":
                    output_table[-1].append(f"{v*57.3:.3f}")
                else:
                    output_table[-1].append(v)
            if k == sort_index:
                sort_values.append(v)
    output_table = np.array(output_table)
    sort_values = np.array(sort_values)
    output_table = output_table[sort_values.argsort()]
    row0 = []
    for key in res[0]:
        row0.append(key)
    output_table = output_table.tolist()
    output_table.insert(0, row0)
    return tabulate.tabulate(output_table, tablefmt='html')

def grid_search(input_data, output_data, gt_data, agent_num, params, params_fixed={}, cmd="roslaunch d2pgo d2pgo_test_multi.launch ", is_root=True):
    #Gird search in param lists
    pg_gt = PoseGraph()
    pg_gt.read_g2o_single(gt_data)
    params_num = len(params)
    #DFS on params
    unique, row = check_param_unique(params)
    if unique:
        #Only one param, no need to grid search
        call_d2pgo(input_data, output_data, agent_num, params, params_fixed, cmd)
        pg_output = PoseGraph()
        pg_output.read_g2o_folder(output_data, prt=False)
        ate_T, ate_rot = ATE(pg_output, pg_gt)
        # align_posegraphs(pg_output, pg_gt)
        # ax = pg_gt.show("GT")
        # pg_output.show("output", ax=ax, clear=False)
        # print(params, ate_T, ate_rot)
        ret_data = {}
        for k, v in params:
            ret_data[k] = v[0]
        ret_data["ATE_T"] = ate_T
        ret_data["ATE_rot"] = ate_rot
        return [ret_data]
    else:
        #More than one param, need to grid search
        param_values = params[row][1]
        res = []
        for i in range(len(param_values)):
            new_params = copy.deepcopy(params)
            new_params[row][1] = [param_values[i]]
            ret = grid_search(input_data, output_data, gt_data, agent_num, new_params, params_fixed, cmd, is_root=False)
            for obj in ret:
                res.append(obj)
    return res

In [ ]:
intput_data = "/home/xuhao/data/pgo/parking-garage/input"
gt_data = "/home/xuhao/data/pgo/parking-garage/groundtruth.g2o"
output_data = "/home/xuhao/data/pgo/parking-garage/output/"
agent_num = 5
params = [
    ["max_steps", [10]],
    ["rho_frame_T", np.logspace(-4, 0, 10, base=10)],
    ["rho_frame_theta", np.logspace(-3, 0, 10, base=10)]
]
ret = grid_search(intput_data, output_data, gt_data, agent_num, params)
print_res_to_table(ret)

In [36]:
input_data = "/home/xuhao/data/d2slam/pgo/tum_corr/input"
gt_data = "/home/xuhao/data/d2slam/pgo/tum_corr/ceres-output.g2o"
output_data = "/home/xuhao/data/d2slam/pgo/tum_corr/output/"
agent_num = 2
params = [
    ["max_steps", [100]],
    ["rho_frame_T", np.logspace(-4, 1, 10, base=2)],
    ["rho_frame_theta", np.logspace(-3, 1, 10, base=2)],
    ["eta_k", np.linspace(0.1, 0.9, 5)]
]
ret = grid_search(input_data, output_data, gt_data, agent_num, params)
print_res_to_table(ret)

max_steps,rho_frame_T,rho_frame_theta,eta_k,ATE_T,ATE_rot
100,0.09185840576722491,0.17009875002179714,0.9,0.311m,1.689°
100,0.09185840576722491,0.2314686780718226,0.9,0.312m,1.949°
100,0.09185840576722491,0.125,0.9,0.316m,1.796°
100,0.0625,0.125,0.9,0.337m,1.897°
100,0.0625,0.17009875002179714,0.7000000000000001,0.338m,1.501°
100,0.0625,0.125,0.7000000000000001,0.341m,1.643°
100,0.09185840576722491,0.2314686780718226,0.7000000000000001,0.381m,2.056°
100,0.09185840576722491,0.125,0.7000000000000001,0.385m,1.854°
100,0.0625,0.17009875002179714,0.9,0.397m,2.254°
100,0.09185840576722491,0.17009875002179714,0.7000000000000001,0.398m,1.910°


In [26]:
#Show the convergence of the results
intput_data = "/home/xuhao/data/d2slam/pgo/torus3D/input/"
gt_data = "/home/xuhao/data/d2slam/pgo/torus3D/groundtruth.g2o"
output_data = "/home/xuhao/data/d2slam/pgo/torus3D/output/"
agent_num = 5
params = [
    ["max_steps", np.logspace(0, 2, 10, base=10, dtype=int)],
    ["debug_rot_init_only", ["true"]]
]
fixed_param = {
    "debug_rot_init_only":"true"
}
ret = grid_search(intput_data, output_data, gt_data, agent_num, params, fixed_param)
print_res_to_table(ret)

max_steps,debug_rot_init_only,ATE_T,ATE_rot
1,true,15.017m,123.294
1,true,15.017m,123.294
2,true,15.017m,96.132
4,true,15.017m,49.443
7,true,15.017m,13.335
12,true,15.017m,11.395
21,true,15.017m,7.348
35,true,15.017m,5.280
59,true,15.017m,5.121
100,true,15.017m,5.179


In [6]:
intput_data = "/home/xuhao/data/d2slam/pgo/torus3D/input/"
gt_data = "/home/xuhao/data/d2slam/pgo/torus3D/groundtruth.g2o"
output_data = "/home/xuhao/data/d2slam/pgo/torus3D/output/"
agent_num = 5
params = [
    ["rho_rot_mat", np.logspace(-3, 1, 10, base=10)]
]
fixed_param = {
    "debug_rot_init_only":"true",
    "max_steps": 20
}
ret = grid_search(intput_data, output_data, gt_data, agent_num, params, fixed_param)
print_res_to_table(ret, "ATE_rot")

rho_rot_mat,ATE_T,ATE_rot
0.021544346900318832,15.017m,5.752
0.46415888336127775,15.017m,5.830
0.05994842503189409,15.017m,5.894
0.0027825594022071257,15.017m,6.472
0.1668100537200059,15.017m,6.534
10.0,15.017m,6.670
1.2915496650148828,15.017m,6.711
0.007742636826811269,15.017m,6.731
3.593813663804626,15.017m,6.783
0.001,15.017m,7.317
